# Development of Emotion and Reasoning in the General Speeches of the United Nations: A text-based machine learning approach
## Script 1: Model Traning, Calculation Centroids & Speech Scoring
### Author: Sarah Franzen

In [54]:
# == Import libraries for data processing and NLP ==

import os
import gensim
from pathlib import Path
from gensim.models import Word2Vec
import numpy as np

from nltk.tokenize import sent_tokenize
from random import shuffle
import nltk
from gensim.utils import simple_preprocess
import joblib
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from scipy.spatial.distance import cosine
import glob
import spacy
from multiprocessing import Pool, freeze_support
import pandas as pd


# === Initialize NLP Tools ===

translator = str.maketrans('', '', punctuation) 
tagger = nltk.perceptron.PerceptronTagger()
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
stemmer = SnowballStemmer("english")

# === Set Working Directory ===

# Prompt user to enter working directory path
#wd = input("Please enter your working directory path (e.g., C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit): ").strip()

# Change to the entered working directory
#try:
   # os.chdir(wd)
    #print(f"Working directory set to: {os.getcwd()}")
#except FileNotFoundError:
   # print("ERROR: The directory you entered does not exist. Please restart and enter a valid path.")
    #exit(1)

# Set your working directory (adjust this as needed)
#wd = r"C:\Users\sarah\OneDrive\Dokumente\Masterarbeit"
#os.chdir(wd)

# Set your working directory (adjust this as needed)
wd = Path(r"C:\Users\sarah\OneDrive\Dokumente\Masterarbeit")

# === Create new Folders ===
additional_folders = ["results", "models"]

# Create/check folders directly in wd
for folder in additional_folders:
    folder_path = Path(wd) / folder
    folder_path.mkdir(exist_ok=True)
    print(f"Folder checked/created: {folder_path}")

# === Define Folder Paths ===

# Folders were already created in the script 0_data_creation
data_c = wd / "data"
data_temp = data_c / "temp"
data_freq = data_c / "freq"
data_dict = data_c / "dictionaries"
data_sent = data_c / "sentences"
data_preprocessed = data_c / "preprocessed"
fig_dir = wd /"fig"

wd_results = wd / "results" # THESE FOLDERS NEED TO BE CREATED
wd_models = wd / "models" # THESE FOLDERS NEED TO BE CREATED

# Upload ressources
stopwords = joblib.load(data_c / "stopwords.pkl")              #### this is from the replication package! Issue for replication
word_counts = joblib.load(data_freq / "word_counts.pkl")
word_counts_weighted = joblib.load(data_freq / "word_counts_weighted.pkl")
affect_dic = joblib.load(data_dict / 'dictionary_affect.pkl')  ### same issue here!
cognition_dic = joblib.load(data_dict / 'dictionary_cognition.pkl') ### same issue here! must also be fixed in 0_data_creation

In [55]:
os.chdir(data_temp)
cleaned_files = [
    str(data_temp / 'clean_speeches_indexed1.pkl'),
    str(data_temp / 'clean_speeches_indexed2.pkl'),
    str(data_temp / 'clean_speeches_indexed3.pkl'),
    str(data_temp / 'clean_speeches_indexed4.pkl')
]

___

### Sentence Split

In [56]:
os.chdir(data_sent)

# Function to split cleaned speeches (clean_speeches) into sentences, tokenize, clean, tag, stem, filter, and save them.

def extract_sentences(dataname):
    data = joblib.load(dataname)
    data = [a[1] for a in data]  # keep only text, no id

    sentences = []
    for doc in data:
        sentences += sent_tokenize(doc)  # use nltk's sent_tokenize here

    sentences = [item for item in sentences if len(item.split()) > 1]
    sentences = [gensim.utils.simple_preprocess(item) for item in sentences]

    sentences = [[a for a in s if not a.isdigit()] for s in sentences]
    sentences = [[a for a in s if len(a) > 2] for s in sentences]

    sentences = [tagger.tag(s) for s in sentences]
    sentences = [[i[0] for i in s if i[1].startswith(('N', 'V', 'J'))] for s in sentences]

    sentences = [[stemmer.stem(i) for i in s] for s in sentences]
    sentences = [[a for a in s if a not in stopwords] for s in sentences]
    sentences = [[a for a in s if word_counts[a] >= 10] for s in sentences]

    sentences = [s for s in sentences if len(s) > 1]
    shuffle(sentences)

    lab = dataname.replace('clean_speeches_', 'sentences_').replace('_.pkl', '.pkl')
    print(f'{dataname} processed')
    joblib.dump(sentences, lab)
    
    unique_tokens = set(token for s in sentences for token in s)
    print(f"Unique tokens: {len(unique_tokens)}")

    print(f'{lab} saved')

for fname in cleaned_files:
    extract_sentences(fname)

C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl processed
Unique tokens: 10778
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed1.pkl saved
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl processed
Unique tokens: 10793
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed2.pkl saved
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed3.pkl processed
Unique tokens: 10797
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed3.pkl saved
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed4.pkl processed
Unique tokens: 10803
C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\sentences_indexed4.pkl saved


In [57]:
sentences_files = [
    os.path.join(data_temp, 'sentences_indexed1.pkl'),
    os.path.join(data_temp, 'sentences_indexed2.pkl'),
    os.path.join(data_temp, 'sentences_indexed3.pkl'),
    os.path.join(data_temp, 'sentences_indexed4.pkl')
]

In [58]:
# == Get sum of unique tokens ==
all_unique_tokens = set()

for dataname in sentences_files:
    data = joblib.load(dataname)  # load list of tokenized sentences
    for sentence in data:
        all_unique_tokens.update(sentence)  # add tokens to the set

print(f"Total unique tokens across all files: {len(all_unique_tokens)}")

sentences_files = [
    'sentences_indexed1.pkl',
    'sentences_indexed2.pkl',
    'sentences_indexed3.pkl',
    'sentences_indexed4.pkl']

Total unique tokens across all files: 10932


---

### Train Word2Vec

In [60]:
dataset = []

for dataname in sentences_files:  
    data = joblib.load(dataname)
    dataset.extend(data) 

# === Model training ===
w2v = Word2Vec(
    sentences=dataset,    
    vector_size=300,      # Dimension of the vector
    window=8,             # Context window size
    min_count=10,         # Minimum word count
    workers=8,            # Number of threads
    sample=1e-3,          # Downsample setting for frequent words
    epochs=10             # Number of iterations over the corpus
)

# Optimize memory usage (optional)
w2v.wv.fill_norms() 

# Save model
wd_models.mkdir(parents=True, exist_ok=True) 
w2v.save(str(wd_models / 'w2v-vectors_8_300.pkl'))

In [61]:
w2v = Word2Vec.load(str(wd_models / "w2v-vectors_8_300.pkl"))
word_vectors = w2v.wv

---

### Calculate Centroids

In [62]:
# == Calculation ==
def findcentroid(text, model):
    vecs = [model.wv[w] * word_counts_weighted[w] for w in text if w in model.wv]
    vecs = [v for v in vecs if len(v) > 0]
    centroid = np.mean(vecs, axis=0)
    #centroid = centroid.reshape(1, -1)
    return centroid


affect_centroid = findcentroid(affect_dic, w2v)
cog_centroid = findcentroid(cognition_dic, w2v)

os.chdir(data_c)
joblib.dump(affect_centroid, 'centroids/affect_centroid.pkl')
joblib.dump(cog_centroid, 'centroids/cog_centroid.pkl')

# == Overview Vectors ==
print("Affect centroid vector:\n", affect_centroid)
print("\nCognition centroid vector:\n", cog_centroid)

# Shape and stats
print("\nShape of affect centroid:", affect_centroid.shape)
print("Shape of cognition centroid:", cog_centroid.shape)

['centroids/cog_centroid.pkl']

---

### Emotionality Scoring

In [65]:
# Set wd to data_preprocessed
os.chdir(data_preprocessed)

# Load preprocessed speech data # NOR SURE IF USING THIS FILE IS CORRECT

preprocessed_final_files = [
   joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed1_final.pkl')),
   joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed2_final.pkl')),
   joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed3_final.pkl')),
   joblib.load(os.path.join(data_preprocessed, 'preprocessed_speeches_indexed4_final.pkl'))
]


In [66]:
# Function to compute weighted document vectors and derive affective/cognitive distances and scores

def documentvecweight(lista):
    out = []
    lista = [i for i in lista if len(i[1]) > 0]
    for s in lista:
        # Compute weighted word vectors for each token present in the Word2Vec model
        vecs = [w2v.wv[w] * word_counts_weighted[w] for w in s[1] if w in w2v.wv]
        if len(vecs) == 0:
            a = np.nan
            c = np.nan
            score = np.nan
        else:
            # Compute mean vector for each speech
            v = np.mean(vecs, axis=0)
             # Cosine distance to affective centroid
            a = cosine(v, affect_centroid)
            # Cosine distance to cognitive centroid
            c = cosine(v, cog_centroid)
            score = (1 + 1 - a) / (1 + 1 - c)
        out.append([s[0], a, c, score])
    return out


def main_function(file_path, idx):
    dataset = joblib.load(file_path)
    data = documentvecweight(dataset)
    lab = os.path.join(data_c, f'distances_main_{idx}.pkl')
    joblib.dump(data, lab)

# Main loop: process all preprocessed speech files
def main():
    files = [
        os.path.join(data_preprocessed, f'preprocessed_speeches_indexed{i+1}_final.pkl') #Changed!
        for i in range(4)
    ]
    for i, f in enumerate(files, start=1):
        main_function(f, i)

if __name__ == "__main__":
    main()

# Merge all distance files into a single DataFrame
DATA_temp = [os.path.join(wd_results, f'distances_main_{i+1}.pkl') for i in range(4)]

tot = []
for dataname in DATA_temp:
    d = joblib.load(dataname)
    tot += d

tot_df = pd.DataFrame(tot, columns=['filename', 'affect_d', 'cognition_d', 'score'])
joblib.dump(tot_df, os.path.join(wd_results, 'distances_10epochs.pkl'))

['C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit\\data\\distances_10epochs.pkl']

In [67]:
# Print the first few rows
print(tot_df.head())

          filename  affect_d  cognition_d     score
0  COL_03_1948.txt  1.159868     0.600979  0.600514
1  HTI_47_1992.txt  0.474285     0.886777  1.370539
2  LKA_46_1991.txt  1.416345     1.189522  0.720137
3  HND_05_1950.txt  0.747536     0.270929  0.724357
4  GHA_25_1970.txt  1.368114     0.735999  0.499909
Shape: (10760, 4)
           affect_d   cognition_d         score
count  10760.000000  10760.000000  10760.000000
mean       1.196353      0.909197      0.738105
std        0.259407      0.201238      0.199796
min        0.293323      0.262322      0.310588
25%        1.032945      0.769638      0.587802
50%        1.235943      0.924176      0.712655
75%        1.392101      1.065732      0.868452
max        1.741876      1.455069      1.613793
              filename  affect_d  cognition_d     score
0      COL_03_1948.txt  1.159868     0.600979  0.600514
1      HTI_47_1992.txt  0.474285     0.886777  1.370539
2      LKA_46_1991.txt  1.416345     1.189522  0.720137
3      HND_05_

In [68]:
# Load df_merged and merge with tot_df by filename 
un_corpus_merged = pd.read_csv(os.path.join(data_c, "un_corpus_merged.csv"), sep=';', encoding='utf-8') 
un_corpus_scored = un_corpus_merged.merge(tot_df, on="filename", how="left")

joblib.dump(un_corpus_scored, os.path.join(data_c, "un_corpus_scored.pkl"))

un_corpus_scored.to_csv(
    os.path.join(wd_results, "un_corpus_scored.csv"),
    sep=';', 
    index=False, 
    encoding='utf-8'
)

In [69]:
print(un_corpus_scored)

              filename                                             speech  \
0      COL_03_1948.txt  Mr. URDANETA-ARBELAEZ declared that when the s...   
1      HTI_47_1992.txt  In the name of the \npeople of Haiti, I am hap...   
2      LKA_46_1991.txt  ﻿I have the honour to convey to you, Sir, and ...   
3      HND_05_1950.txt  It is indeed difficult to find words which do ...   
4      GHA_25_1970.txt  121.\t I should like to begin by congratulatin...   
...                ...                                                ...   
10755  SLV_66_2011.txt  \nThis new session of the Assembly of the Unit...   
10756  AUT_23_1968.txt  1. May I ask you, Sir, to convey to the Presid...   
10757  CIV_24_1969.txt  83. Permit me to take this opportunity to pay ...   
10758  CYP_68_2013.txt  It is a great honour to \naddress the General ...   
10759  MWI_34_1979.txt  ﻿Allow me, Mr. President, first to join the re...   

      country_code  year   country_name  speech_length_words  \
0          

In [70]:
# Count where affect_d is NaN
nan_count = un_corpus_scored['affect_d'].isna().sum()

# Count where affect_d is not NaN
not_nan_count = un_corpus_scored['affect_d'].notna().sum()

print("Count where affect_d is NaN:", nan_count)
print("Count where affect_d is not NaN:", not_nan_count)

Count where affect_d is NaN: 0
Count where affect_d is not NaN: 10760
